In [ ]:
import numpy as np
import random
import copy
import torch
import torch.nn.functional as F
import torch.optim as optim

In [20]:
from collections import namedtuple, deque

In [23]:
class Agent():
    
    #initialize an Agent object
    def __init__(self, state_size, action_size, num_agents, random_seed):
        
        #set dimension of state vector size
        self.state_size = state_size
        
        #set dimension of action vector size
        self.action_size = action_size
        
        #set number of agents (cores or workers)
        self.num_agents = num_agents
        
        #set to reproduce results
        self.seed = random.seed(random_seed)
        
        # Actor Network (w/ Target Network)
        self.actor_local = Actor(state_size, action_size, random_seed).to(device)
        self.actor_target = Actor(state_size, action_size, random_seed).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)
        
        # Critic Network (w/ Target Network)
        self.critic_local = Critic(state_size, action_size, random_seed).to(device)
        self.critic_target = Critic(state_size, action_size, random_seed).to(device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY)

        # Add noise to the actions of the agent to ensure exploration using Ornstein Uhlenbeck noise process
        self.noise = OUNoise((num_agents, action_size), random_seed)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)
    
    
    
    def step(self, states, actions, rewards, next_states, dones):
        
        # Save experience / reward
        for agent in range(self.num_agents):
            self.memory.add(states[agent,:], actions[agent,:], rewards[agent], next_states[agent,:], dones[agent])
        
        # Learn, if enough samples are available in memory
        if len(self.memory) > BATCH_SIZE:
            
            #Grab a batch of transitions from memory
            experiences = self.memory.sample()
            
            #Here BEGINS STEP 3! We will learn using our batch of transitions!
            self.learn(experiences)
            
            
    #Returns actions for given state as per current policy.
    def act(self, state, add_noise=True):
        
        #we prepare our state
        state = torch.from_numpy(state).float().to(device)
        
        #we create an empty action matrix
        acts = np.zeros((self.num_agents, self.action_size))
        
        #To set NN in test mode instead of train mode
        self.actor_local.eval()
        
        #again, we don't want to calculate gradients because we are only testing!
        with torch.no_grad():
            for agent in range(self.num_agents):
                
                #forward propagate states in actor local NN to get actions!
                acts[agent,:] = self.actor_local(state[agent,:]).cpu().data.numpy()
        
        #This is to set the model back to training mode, but no training is done here yet!
        self.actor_local.train()
        
        #adding noise for exploration!
        if add_noise:
            acts += self.noise.sample()
        return np.clip(acts, -1, 1)
    
    
    def reset(self):
        self.noise.reset()

        
    def learn(self, experiences):
        #  Update policy and value parameters using given batch of experience tuples.
        #  Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        #  where:
        #      actor_target(state) -> action
        #      critic_target(state, action) -> Q-value
        #  Params
        #  ======
        #      experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
        #      gamma (float): discount factor
        
        #  states, actions, rewards, next_states, dones = experiences

        # ---------------------------- update critic ---------------------------- #
        # Get predicted next-state actions and Q values from target models
        states, actions, rewards, next_states, dones = experiences
        
        actions_next = self.actor_target(next_states) 
        Q_targets_next = self.critic_target(next_states, actions_next)
        
        # Compute Q targets for current states (y_i)
        """12. calculate 'yi' """
        Q_targets = rewards + (GAMMA * Q_targets_next * (1 - dones))
        
        # Compute critic loss
        Q_expected = self.critic_local(states, actions)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        
        #torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), 1)
        self.critic_optimizer.step()
        
        
        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        actions_pred = self.actor_local(states)
        actor_loss = -self.critic_local(states, actions_pred).mean()
        # Minimize the loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # ----------------------- update target networks ----------------------- #
        self.soft_update(self.critic_local, self.critic_target, TAU)
        self.soft_update(self.actor_local, self.actor_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [ ]:
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0.0, theta=0.15, sigma=0.15, sigma_min = 0.05, sigma_decay=.975):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.sigma_min = sigma_min
        self.sigma_decay = sigma_decay
        self.seed = random.seed(seed)
        self.size = size
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)
        """Resduce  sigma from initial value to min"""
        self.sigma = max(self.sigma_min, self.sigma*self.sigma_decay)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.standard_normal(self.size)
        self.state = x + dx
        return self.state